In [ ]:
from virusPropagationModel import *
import VPM_plotting as vpm_plot
%matplotlib inline
%load_ext autoreload
import glob
import os
import copy
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.cm as cm
import numpy as np
#%load_ext memory_profiler

In [ ]:
geopath = 'datafiles/'
geofiles = {0:'Buildings_Gangelt_MA_1.csv',
            1:'Buildings_Gangelt_MA_3.csv',
           }         

# Build world 

### Choose which world to initate

In [ ]:
#world_to_pick = 0 ## choosing large version
world_to_pick = 1 ## choosing small version

### Initiate the selected world without infections

In [ ]:
modeledWorld = ModeledPopulatedWorld(1000, 0, world_from_file=True, geofile_name=geopath+geofiles[world_to_pick], agent_agent_infection=True, input_schedules='schedules_v2')
n_people = modeledWorld.number_of_people
print('Number of agents: '+ str(n_people))

### Definition of the function to infect specified susceptible agents

In [ ]:
def infect_world(world, IDs=[1]):
    ID_list = world.get_remaining_possible_initial_infections(IDs)
    world.initialize_infection(specific_people_ids=ID_list)

# Run initial simulation

#### Define Simulation characteristics

In [ ]:
timesteps_initial_sim=1500
general_infectivity=0.3
general_interaction_frequency=2

#### Copy the initialized world and infect n agents in that copy

In [ ]:
world_to_use = copy.deepcopy(modeledWorld)

n_initially_infected=4

infect_world(world_to_use, IDs=[i+1 for i in range(n_initially_infected)])

#### Set-up simulation with specified characteristics

In [ ]:
simulation_initial_wave = Simulation(world_to_use, timesteps_initial_sim, run_immediately=False)

simulation_initial_wave.change_agent_attributes({'all':{'behaviour_as_infected':{'value':general_infectivity,'type':'replacement'}}})
simulation_initial_wave.interaction_frequency=general_interaction_frequency

#### Run simulation

In [ ]:
simulation_initial_wave.simulate()

#### Plot status-trajectories

In [ ]:
simulation_initial_wave.plot_status_timecourse()

# Derive ordered lists of agents for different vaccination-strategies

## Sorting-strategies, based on previous simulation

### Sorted by forecasted infection wave

#### Define function to sort agents according to forecasted infection wave

In [ ]:
def get_previous_infections(previous_simulation):
    df_inf = previous_simulation.get_infection_event_information()
    infected = list(df_inf['h_ID'].values)
    all_agents = infected+[a.ID for a in previous_simulation.people if a.ID not in infected]
    return all_agents

#### Create list of agents sorted by forecasted infection wave

In [ ]:
agents_forecasted = get_previous_infections(simulation_initial_wave)

### Sorted by over-representation (agent types)

#### Define function to sort agents by type. The sequence of types is defined in sorted list "agent_types". 

In [ ]:
def get_ids_by_agent_type(world):
    agent_types = ['under_age', 'adult', 'teacher', 'medical_professional', 'public_worker', 'pensioner']
    ids_by_type = {s_type:[] for s_type in agent_types}
    for p in world.people:
        ids_by_type[p.schedule['type']].append(p.ID)
    ordered_ids = []
    for s_type in agent_types:
        ordered_ids.extend(ids_by_type[s_type])
    return ordered_ids

#### Create list of agents sorted by type

In [ ]:
agents_by_type = get_ids_by_agent_type(modeledWorld)

### Sorted by Interactivity

#### Define function to sort agents with decreasing number of total interactions from a preceding simulation without infection spreading

In [ ]:
def get_ids_by_interactions(simulation_no_infections):
    contacts = simulation_no_infections.get_contact_distributions(min_t=0, max_t=168)[1]
    contacts_mean = contacts.groupby('ID').mean()
    contacts_mean.reset_index(inplace=True)
    contacts_sorted = contacts_mean.sort_values('interactions', axis=0, ascending=False)
    to_recover_list = list(contacts_sorted['ID'].values)
    return to_recover_list

#### Run the preceding simulation without infection spreading

In [ ]:
uninfectedWorld = copy.deepcopy(modeledWorld)
simulation_no_infections = Simulation(uninfectedWorld, 170, run_immediately=False)
simulation_no_infections.interaction_frequency=general_interaction_frequency
simulation_no_infections.simulate()

#### Create the list of agents sorted by decreasing number of total interactions

In [ ]:
agents_by_interactions = get_ids_by_interactions(simulation_no_infections)

## Sorting-strategies, based on population-structure

### Sorted by household

#### Define function to sort agents by their households. First one member of each household is chosen. When there is one member for each household a second member is chosen for each household of size 2 and bigger. After that a third member is chosen for each household of size 3 and bigger. ...

In [ ]:
def get_ids_by_households(world):
    ai = world.get_agent_info()
    home_count_dict = dict(zip(list(ai.groupby('Home').count().index), list(ai.groupby('Home').count()['ID'])))
    ai['Home_size']=ai['Home'].map(home_count_dict)

    ai_sorted = ai.sort_values('Home')
    home_list = ai_sorted['Home'].values

    count_list=[]
    predecessor = 0
    count = 1
    for x in home_list:
        if x == predecessor:
            count +=1
        else:
            count = 1
        count_list.append(count)
        predecessor = x

    ai_sorted['Home_position'] = count_list

    ai_sorted.sort_values('Home_position') ## dataframe
    households_sorted = ai_sorted.sort_values(by=['Home_position'])
    to_recover_list = list(households_sorted['ID']) ### id list
    return to_recover_list

#### Create the list of agents sorted by households

In [ ]:
agents_by_households = get_ids_by_households(modeledWorld)

### Sorted by age

#### Define function to sort agents by age, descending

In [ ]:
def get_ids_by_age(world):
    ages_and_ids = [(p.age, p.ID) for p in world.people]
    ages_and_ids_df = pd.DataFrame(ages_and_ids, columns=['age','id'])
    ages_and_ids_df.sort_values('age', ascending=False, inplace=True)
    to_recover_list = list(ages_and_ids_df['id'])
    return to_recover_list

#### Create the list of agents sorted by age, descending

In [ ]:
agents_by_age = get_ids_by_age(modeledWorld)

## Random sorting

#### Define function to randomly shuffle the order of agents

In [ ]:
def get_random_id_list(world):
    agent_ids = [p.ID for p in world.people]
    random.shuffle(agent_ids)
    return agent_ids

#### Create a list of agents with random order

In [ ]:
agents_random = get_random_id_list(modeledWorld)

# Run vaccination-fraction screens for different strategies

#### Define function to vaccinate a certain fraction of agents in a specified order

In [ ]:
# define function to vaccinate list of agents
def vaccinate_world(world, frac, ordered_agents):
    agents_to_vaccinate = ordered_agents[:int(frac*len(ordered_agents))]
    for p in world.people:
        if p.ID in agents_to_vaccinate:
            p.set_initially_recovered()

#### Create a dictionary to access the previously created ordered lists by keywords

In [ ]:
agent_orders = {'forecasted':agents_forecasted, 'overrepresentation':agents_by_type,
                'interactions':agents_by_interactions, 'households':agents_by_households,
                'age':agents_by_age, 'random':agents_random}

#### Define the fractions of agents to vaccinate

In [ ]:
fractions = np.arange(0.1, 0.91, 0.2)

#### Define function to simulate the vaccination scenario with the defined fractions and agent orders

In [ ]:
def simulate_fractions(modeledWorld, ordered_agents, fractions, n_initially_infected=4, timesteps_vaccination_sim=3000):
    results=pd.DataFrame(columns=['fraction','infected','icued','dead'])
    for frac in fractions:
        world_to_use = copy.deepcopy(modeledWorld)
        vaccinate_world(world_to_use, frac, ordered_agents)
        infect_world(world_to_use, IDs=[i+1 for i in range(n_initially_infected)])
        simulation_vaccinated_fraction = Simulation(world_to_use, timesteps_vaccination_sim, run_immediately=False)
        simulation_vaccinated_fraction.change_agent_attributes({'all':{'behaviour_as_infected':{'value':general_infectivity,'type':'replacement'}}})
        simulation_vaccinated_fraction.interaction_frequency=general_interaction_frequency
        simulation_vaccinated_fraction.simulate()
        cummulative_flags = simulation_vaccinated_fraction.get_flag_sums_over_time(specific_flags=['ICUed'])
        dead_agents = simulation_vaccinated_fraction.get_status_trajectories(specific_statuses=['D'])['D']['D'].iloc[-1]
        infected_agents = simulation_vaccinated_fraction.get_infection_event_information().shape[0]
        icued_agents = cummulative_flags['ICUed'].max()
        results.loc[frac,'fraction']=frac
        results.loc[frac,'infected']=infected_agents
        results.loc[frac,'icued']=icued_agents
        results.loc[frac,'dead']=dead_agents
    return results

#### Define a list of strategies to test by keywords, run the simulations for all fractions and all strategies and save relevant results

In [ ]:
strategies_to_test = ['forecasted', 'overrepresentation', 'interactions', 'households', 'age', 'random']
results_dict = {strategy: simulate_fractions(modeledWorld, agent_orders[strategy], fractions) for strategy in strategies_to_test}

# Plot results of vaccination screens

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(15,5))

colors = {'random':'lightseagreen', 'interactions':'violet', 'age':'gold',
          'overrepresentation':'royalblue', 'households':'lightgreen', 'forecasted':'orange'}

for strategy in results_dict:
    fractions = results_dict[strategy]['fraction']
    axs[0].plot(fractions, results_dict[strategy]['infected']/(modeledWorld.number_of_people*(1-fractions)),
                label=strategy, lw=2, alpha=0.7, color=colors[strategy])
    axs[1].plot(fractions, results_dict[strategy]['dead']/(modeledWorld.number_of_people*(1-fractions)),
                label=strategy, lw=2, alpha=0.7, color=colors[strategy])
    axs[2].plot(fractions, (results_dict[strategy]['icued']/modeledWorld.number_of_people)*100000.0,
                label=strategy, lw=2, alpha=0.7, color=colors[strategy])
    
for ax in axs:
    ax.set_xlabel('fraction of immunized population')
    
axs[0].set_ylabel('fraction of non immunized population')
axs[1].set_ylabel('fraction of non immunized population')
axs[2].set_ylabel('max ICU demand per 100.000')

axs[0].set_title('Infections')
axs[1].set_title('Death')
axs[2].set_title('ICU Demand')

axs[0].legend()

plt.tight_layout()